In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Physical_Exercise_Response/GSE111553'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Blood and skeletal muscle gene signatures of exercise training in men (Total blood data set G1)"
!Series_summary	"Accounting for transcriptional features of endurance adaptations to training is important and could help elucidate the high variability in oxygen uptake (VO2) response. We aimed to identify whole-transcriptome signatures of an endurance training protocol in whole-blood (leukocytes), PBMCs and skeletal muscle tissue of the same group of individuals in a controlled environment."
!Series_overall_design	"Two groups of healthy untrained policemen recruits (G1, n=20, 26±4 yrs; G2, n= 21, 27±3 yrs) were selected. Blood (PAXgene and EDTA tubes) and muscle tissueforgene expression assays and peak VO2peak measures by cardiopulmonary exercise test were taken before and after 18 weeks of running (3 times/wk; 60min). RNA was extracted from whole-blood (leukocytes), PBMCs and muscle and was used for whole-genome expression analysis using AffymetrixG

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Evaluate if 'tissue' can be used as a proxy for 'Physical_Exercise_Response'
trait_row = 0  # Attempt to link 'tissue' to 'Physical_Exercise_Response'
age_row = 2
gender_row = 1

def convert_trait(value):
    if ':' in value:
        value = value.split(':')[-1].strip()
    # Example conversion: total blood (leukocytes) -> 1, otherwise -> 0
    if value.lower() == 'total blood (leukocytes)':
        return 1
    else:
        return 0

def convert_age(value):
    if ':' in value:
        value = value.split(':')[-1].strip()
    try:
        return float(value) if value.replace('.', '', 1).isdigit() else None
    except ValueError:
        return None

def convert_gender(value):
    if ':' in value:
        value = value.split(':')[-1].strip().lower()
    if value == 'male':
        return 1
    elif value == 'female':
        return 0
    else:
        return None

save_cohort_info('GSE111553', './preprocessed/Physical_Exercise_Response/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Physical_Exercise_Response', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Physical_Exercise_Response/trait_data/GSE111553.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3034095': [1.0, 27.0, 1.0], 'GSM3034096': [1.0, 27.0, 1.0], 'GSM3034097': [1.0, 30.0, 1.0], 'GSM3034098': [1.0, 23.0, 1.0], 'GSM3034099': [1.0, 23.0, 1.0], 'GSM3034100': [1.0, 30.0, 1.0], 'GSM3034101': [1.0, 22.0, 1.0], 'GSM3034102': [1, None, 1], 'GSM3034103': [1.0, 29.0, 1.0], 'GSM3034104': [1.0, 29.0, 1.0], 'GSM3034105': [1.0, 25.0, 1.0], 'GSM3034106': [1.0, 26.0, 1.0], 'GSM3034107': [1.0, 23.0, 1.0], 'GSM3034108': [1.0, 28.0, 1.0], 'GSM3034109': [1.0, 28.0, 1.0], 'GSM3034110': [1.0, 27.0, 1.0], 'GSM3034111': [1.0, 30.0, 1.0], 'GSM3034112': [1.0, 28.0, 1.0], 'GSM3034113': [1.0, 19.0, 1.0], 'GSM3034114': [1.0, 30.0, 1.0], 'GSM3034115': [1.0, 27.0, 1.0], 'GSM3034116': [1.0, 26.0, 1.0], 'GSM3034117': [1.0, 27.0, 1.0], 'GSM3034118': [1.0, 27.0, 1.0], 'GSM3034119': [1.0, 30.0, 1.0], 'GSM3034120': [1.0, 23.0, 1.0], 'GSM3034121': [1.0, 23.0, 1.0], 'GSM3034122': [1.0, 30.0, 1.0], 'GSM3034123': [1.0, 22.0, 1.0], 'GSM3034124': [1, None, 1], 'GSM3034125': [1.0, 29.0, 1.0], 'GSM3034126': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st',
       '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st',
       '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st',
       '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST0000040838

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the correct keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'gene_assignment'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to the gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
